In [1]:
import numpy as np

In [2]:
class Perceptron:
    """ A single neuron with the sigmoid activate function
        Attributtes:
            inputs: The number of inputs in the perceptron, not cunting the bias
            bias: the bias term. By default it´s 1.0
    """
    
    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs +1 (for the bias)"""
        self.weights = np.random.rand(inputs+1)*2 -1
        self.bias = bias
        
    def run(self,x):
        """Run the perceptron. x is a python list with the input values."""
        x_sum = np.dot(np.append(x,self.bias),self.weights)
        #this calculates the product point of the inputs and the weights
        return self.sigmoid(x_sum)
    
    def set_weights(self,w_init):
        """Set the weights. w_init is a python list with the weights"""
        self.weights = np.array(w_init)

    
    def sigmoid(self,x):
        """Evaluate the sigmoid function for thw floating point input x"""
        return 1/(1 + np.exp(-x))

In [3]:

class MultiLayerPerceptron:
    """A multilayer perceptron class that uses the perceptron class above.
        Attributtes:
            layers: A python list with the number of elements per layer
            bias: The bias term. The same bias is used for all neurons
            eta: The learning rate"""
    def __init__(self,layers,bias = 1.0, eta = 0.5):
        """Return a new MLP object with the specified parameters"""
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] #the list of all neurons
        self.values = [] #the list of all output values
        self.d= []
        
        for i in range(len(self.layers)):
            self.values.append([])
            self.d.append([])
            self.network.append([])
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d[i] = [0.0 for j in range(self.layers[i])]
            if i>0:
                for j in range(self.layers[i]):
                    self.network[i].append(Perceptron(inputs=self.layers[i-1],bias=self.bias))
    
        self.network = np.array([np.array(x) for x in self.network],dtype=object)
        self.values=np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)
    
    def set_weights(self,w_init):
        """set the weights.
            w_init is a list of lists with the weights for all, but the input layer"""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j])
                
    def printWeights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].weights)
            print()
            
    def run(self,x):
        """Feed a sample x into the multilayer perceptron"""
        x=np.array(x,dtype=object)
        self.values[0]=x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.values[i][j]= self.network[i][j].run(self.values[i-1])
        return self.values[-1]
        
    #Backpropagation method
    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorythm)."""
        x = np.array(x, dtype=object)
        y = np.array(y, dtype=object)

        #Step 1:  feed a sample to the network
        outputs = self.run(x)

        #Step 2: calculate the MSE
        error = (y-outputs)
        MSE = sum(error ** 2)/ self.layers[-1]

        #Step 3: Calculate the output error terms
        self.d[-1] = outputs * (1-outputs)*error

        #Step 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]):
                    fwd_error+= self.network[i+1][k].weights[h] * self.d[i+1][k]
                self.d[i][h]=self.values[i][h] * (1-self.values[i][h]) * fwd_error

        #step 5 & 6 : calculate the deltas and update the weights
        #iterates layers
        for i in range(1, len(self.network)):
            #iterates neurons
            for j in range(self.layers[i]):
                #iterates inputs
                for k in range(self.layers[i-1]+1):
                    if k == self.layers[i-1]:
                        delta = self.eta * self.d[i][j] *self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k]+=delta

        return MSE



In [4]:
#test code
mlp = MultiLayerPerceptron(layers=[2,2,1])
print("Training neural networks as an XOR gate \n")
for i in range(3000):
    mse=0.0
    mse+=mlp.bp([0,0],[0])
    mse+=mlp.bp([0,1],[1])
    mse+=mlp.bp([1,0],[1])
    mse+=mlp.bp([1,1],[0])
    mse = mse/4
    if(i%100 == 0):
        print(mse)
        
mlp.printWeights()

print("MLP:")
print("0 0 = {0:.10f}".format(mlp.run([0,0])[0]))
print("0 1 = {0:.10f}".format(mlp.run([0,1])[0]))
print("1 0 = {0:.10f}".format(mlp.run([1,0])[0]))
print("1 1 = {0:.10f}".format(mlp.run([1,1])[0]))

Training neural networks as an XOR gate 

0.2707328017146026
0.2645924002509357
0.26380612695686995
0.26306668471077627
0.2624022437826665
0.26181714261251426
0.2612852670283703
0.26076620615016255
0.2602005402629963
0.2594633370641857
0.258155699922761
0.25471139193572045
0.2447182760239549
0.22431141392152976
0.20237926078221777
0.1886262099341123
0.1795547485146825
0.1696066292124495
0.14884088100319248
0.10024895020600852
0.049273100328090264
0.025961455880009893
0.01622236181129758
0.011405770068928804
0.008649916202985523
0.006901571000041712
0.005707620068594713
0.0048467810544767795
0.004199913882959844
0.003697823531762134

Layer 2 Neuron 0 [-3.84774915 -3.84788153  5.64692394]
Layer 2 Neuron 1 [-5.92131072 -5.95368863  2.21772293]

Layer 3 Neuron 0 [ 7.56050448 -7.85012787 -3.44721102]

MLP:
0 0 = 0.0477536623
0 1 = 0.9456835593
1 0 = 0.9453873895
1 1 = 0.0701471579
